# Dataset settings

In [ ]:
C = 3
p_limit = 1
EPOCHS=60

In [ ]:
input_names = '2-3 4-6 7-9 10-15 20-25 30-35 40-50'.split()

In [ ]:
hp0 = {
    'input_shape': (15*15,),
    'activation': 'relu',
    'first_dense_units': 500,
    'second_dense_units': 1100,
    'third_dense_units': 500,
    'kernel_initializer': 'lecun_uniform',
    'learning_rate': 0.0005875310943952402,
    'optimizer': 'nadam',
    'dropout_ratio': 0.01,
    'loss': 'mse'
}

# Data downloading and organizing

In [ ]:
import os

os.mkdir('./logs')
os.mkdir('./logs/neural')

In [ ]:
os.mkdir('./figs')

# Useful functions:
* bulding model for `super_model architecture`
* OneCycleScheduler
* data preparation
* finding `learning_rate`

In [ ]:
PI_RESOLUTION = 50


def prepare_inputs(pi_resolution):
    model_inputs = {
        'current': slice(0, 1),
    }

    i = 0
    for k in range(3):
        for p in ['a', 'b', 'c']:
            model_inputs[f'b{k}_p{p}'] = slice(i * pi_resolution ** 2 + 1, (i + 1) * pi_resolution ** 2 + 1)
            i += 1

    return model_inputs

In [ ]:
import tensorflow as tf


class OneCycleScheduler(tf.keras.callbacks.Callback):
    def __init__(self, iterations, max_lr=1e-3, start_lr=None,
                 last_iterations=None, last_lr=None):
        self.iterations = iterations
        self.max_lr = max_lr
        self.start_lr = start_lr or max_lr / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_lr = last_lr or self.start_lr / 1000
        self.iteration = 0

    def _interpolate(self, iter1, iter2, lr1, lr2):
        return (lr2 - lr1) * (self.iteration - iter1) / (iter2 - iter1) + lr1

    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            lr = self._interpolate(0, self.half_iteration, self.start_lr,
                                   self.max_lr)
        elif self.iteration < 2 * self.half_iteration:
            lr = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                   self.max_lr, self.start_lr)
        else:
            lr = self._interpolate(2 * self.half_iteration, self.iterations,
                                   self.start_lr, self.last_lr)
        self.iteration += 1
        self.model.optimizer.learning_rate = lr

In [ ]:
current_lr = 3.2e-05
from keras.layers import Dropout, AlphaDropout

def build_drop_super_model_kt(hp):
    """Function that builds `super_model_architecture`"""
    # hyperparams
    
    _input_shape = hp["input_shape"]
    _activation = hp["activation"]
    first_dense_neurons = hp["first_dense_units"]
    second_dense_neurons = hp["second_dense_units"]
    third_dense_neurons = hp["third_dense_units"]

    _first_dense_units = first_dense_neurons
    _second_dense_units = second_dense_neurons
    _third_dense_units = third_dense_neurons

    _kernel_initializer = hp["kernel_initializer"]
    learning_rate = hp["learning_rate"]
    optimizer = hp["optimizer"]
    _dropout_rate = hp["dropout_ratio"]

    if 'adam' == optimizer:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    else:
        optimzier = tf.keras.optimizers.Nadam(learning_rate=learning_rate)

    global current_lr
    current_lr = learning_rate

    if _activation in ('lecun_normal', 'lecun_uniform'):
        _dropout = AlphaDropout
    else:
        _dropout = Dropout

    # input layers
    b0_pa = tf.keras.layers.Input(shape=_input_shape, name='b0_pa')
    b0_pb = tf.keras.layers.Input(shape=_input_shape, name='b0_pb')
    b0_pc = tf.keras.layers.Input(shape=_input_shape, name='b0_pc')

    b1_pa = tf.keras.layers.Input(shape=_input_shape, name='b1_pa')
    b1_pb = tf.keras.layers.Input(shape=_input_shape, name='b1_pb')
    b1_pc = tf.keras.layers.Input(shape=_input_shape, name='b1_pc')

    b2_pa = tf.keras.layers.Input(shape=_input_shape, name='b2_pa')
    b2_pb = tf.keras.layers.Input(shape=_input_shape, name='b2_pb')
    b2_pc = tf.keras.layers.Input(shape=_input_shape, name='b2_pc')

    # first dense
    b0_pa_next = _dropout(_dropout_rate)(tf.keras.layers.Dense(units=_first_dense_units, activation=_activation,
                                                               kernel_initializer=_kernel_initializer,
                                                               name='b0_pa_dense')(b0_pa))
    b0_pb_next = _dropout(_dropout_rate)(tf.keras.layers.Dense(units=_first_dense_units, activation=_activation,
                                                               kernel_initializer=_kernel_initializer,
                                                               name='b0_pb_dense')(b0_pb))
    b0_pc_next = _dropout(_dropout_rate)(tf.keras.layers.Dense(units=_first_dense_units, activation=_activation,
                                                               kernel_initializer=_kernel_initializer,
                                                               name='b0_pc_dense')(b0_pc))
    b1_pa_next = _dropout(_dropout_rate)(tf.keras.layers.Dense(units=_first_dense_units, activation=_activation,
                                                               kernel_initializer=_kernel_initializer,
                                                               name='b1_pa_dense')(b1_pa))
    b1_pb_next = _dropout(_dropout_rate)(tf.keras.layers.Dense(units=_first_dense_units, activation=_activation,
                                                               kernel_initializer=_kernel_initializer,
                                                               name='b1_pb_dense')(b1_pb))
    b1_pc_next = _dropout(_dropout_rate)(tf.keras.layers.Dense(units=_first_dense_units, activation=_activation,
                                                               kernel_initializer=_kernel_initializer,
                                                               name='b1_pc_dense')(b1_pc))
    b2_pa_next = _dropout(_dropout_rate)(tf.keras.layers.Dense(units=_first_dense_units, activation=_activation,
                                                               kernel_initializer=_kernel_initializer,
                                                               name='b2_pa_dense')(b2_pa))
    b2_pb_next = _dropout(_dropout_rate)(tf.keras.layers.Dense(units=_first_dense_units, activation=_activation,
                                                               kernel_initializer=_kernel_initializer,
                                                               name='b2_pb_dense')(b2_pb))
    b2_pc_next = _dropout(_dropout_rate)(tf.keras.layers.Dense(units=_first_dense_units, activation=_activation,
                                                               kernel_initializer=_kernel_initializer,
                                                               name='b2_pc_dense')(b2_pc))

    # first concatenations
    c0 = tf.keras.layers.Concatenate(name='concat_b0')([b0_pa_next, b0_pb_next, b0_pc_next])
    c1 = tf.keras.layers.Concatenate(name='concat_b1')([b1_pa_next, b1_pb_next, b1_pc_next])
    c2 = tf.keras.layers.Concatenate(name='concat_b2')([b2_pa_next, b2_pb_next, b2_pc_next])

    # second dense
    c0_next = _dropout(_dropout_rate)(tf.keras.layers.Dense(units=_second_dense_units, activation=_activation,
                                                            kernel_initializer=_kernel_initializer, name='c0_dense')(
        c0))
    c1_next = _dropout(_dropout_rate)(tf.keras.layers.Dense(units=_second_dense_units, activation=_activation,
                                                            kernel_initializer=_kernel_initializer, name='c1_dense')(
        c1))
    c2_next = _dropout(_dropout_rate)(tf.keras.layers.Dense(units=_second_dense_units, activation=_activation,
                                                            kernel_initializer=_kernel_initializer, name='c2_dense')(
        c2))

    # second concatenations
    current = tf.keras.layers.Input(shape=(1,), name='current')
    c_last = tf.keras.layers.Concatenate(name='concat_last')([current, c0_next, c1_next, c2_next])

    # third dense
    c_last_dense = tf.keras.layers.Dense(units=_third_dense_units, activation=_activation,
                                         kernel_initializer=_kernel_initializer, name='c_last_dense')(c_last)
    # output
    out = tf.keras.layers.Dense(units=1, name='output')(c_last_dense)

    model = tf.keras.Model(inputs=[current, b0_pa, b0_pb, b0_pc, b1_pa, b1_pb, b1_pc, b2_pa, b2_pb, b2_pc],
                           outputs=[out])

    model.compile(loss=hp["loss"],
                  optimizer=optimizer,
                  metrics=[tf.keras.metrics.MeanAbsoluteError(), tf.keras.metrics.MeanSquaredError()]
                  )
    return model

# Data preparation

In [ ]:
os.listdir('../input')

In [ ]:
from sklearn.utils import shuffle
from pathlib import Path
from time import strftime
import pickle
import pandas as pd
import math
import time

In [ ]:
best_evals = dict()

In [ ]:
from tensorflow.keras import backend as K

def fine_tune(dataset_name, res):
    if res in {2, 3}:
        path_name = input_names[0]
    elif res in {4, 5, 6}:
        path_name = input_names[1]
    elif res in {7, 8, 9}:
        path_name = input_names[2]
    elif res in {10, 15}:
        path_name = input_names[3]
    elif res in {20, 25}:
        path_name = input_names[4]
    elif res in {30, 35}:
        path_name = input_names[5]
    elif res in {40, 50}:
        path_name = input_names[6]
    else:
        path_name = 'ERROR'
        
    datapath = f'../input/neumann-v3-{path_name}/{dataset_name}__400_data_train.pkl'
    targetpath = f'../input/neumann-v3-{path_name}/{dataset_name}__400_target_train.pkl'

    data = pd.read_pickle(datapath).to_numpy()
    target = pd.read_pickle(targetpath).to_numpy().ravel()

    X_valid = pd.read_pickle(datapath.replace('train', 'test')).to_numpy()
    y_valid = pd.read_pickle(targetpath.replace('train', 'test')).to_numpy().ravel()
    
    X_train, y_train = shuffle(data, target, random_state=42)
    X_valid, y_valid = shuffle(X_valid, y_valid, random_state=42)

    inputs_slices = prepare_inputs(res)

    input_data = [
        X_train[:, inputs_slices['current']],
        X_train[:, inputs_slices['b0_pa']],
        X_train[:, inputs_slices['b0_pb']],
        X_train[:, inputs_slices['b0_pc']],
        X_train[:, inputs_slices['b1_pa']],
        X_train[:, inputs_slices['b1_pb']],
        X_train[:, inputs_slices['b1_pc']],
        X_train[:, inputs_slices['b2_pa']],
        X_train[:, inputs_slices['b2_pb']],
        X_train[:, inputs_slices['b2_pc']],
    ]

    input_valid = [
        X_valid[:, inputs_slices['current']],
        X_valid[:, inputs_slices['b0_pa']],
        X_valid[:, inputs_slices['b0_pb']],
        X_valid[:, inputs_slices['b0_pc']],
        X_valid[:, inputs_slices['b1_pa']],
        X_valid[:, inputs_slices['b1_pb']],
        X_valid[:, inputs_slices['b1_pc']],
        X_valid[:, inputs_slices['b2_pa']],
        X_valid[:, inputs_slices['b2_pb']],
        X_valid[:, inputs_slices['b2_pc']],
    ]

    assert all(x is not None for x in input_data), "Found None in input_data"
    
    if not os.path.exists(f'./logs/neural/{dataset_name}'):
        os.mkdir(f'./logs/neural/{dataset_name}')

    def get_run_logdir(root_logdir=f'./logs/neural/{dataset_name}'):
        return Path(root_logdir) / strftime("run_%Y_%m_%d_%H_%M_%S")

    n_epochs = EPOCHS
    
    hyperparameters_set = [hp0, hp0, hp0, hp0, hp0]
    
    model_eval = []
    for idx, hp in enumerate(hyperparameters_set):
        tb = tf.keras.callbacks.TensorBoard(log_dir=get_run_logdir(f'./logs/neural/{dataset_name}_{idx}'))
                                     
        model = build_drop_super_model_kt(hp)
        
        
        onecycle = OneCycleScheduler(math.ceil(X_train.shape[0] / 32) * n_epochs, max_lr=current_lr)
        history = model.fit(input_data, y_train,
                  epochs=n_epochs,
                  callbacks=[onecycle, tb])
        
        model.save(f'{dataset_name}_{idx}.keras')

        model_eval.append(model.evaluate(input_valid, y_valid))

    global best_evals
    best_evals[dataset_name] = model_eval
    
    K.clear_session()
    with open(f'./working/checkpoint__{int(time.time())}.pkl', 'wb') as fn:
        pickle.dump(best_evals, fn)

In [ ]:
for C in [3]:
    for p in [1]:
        for res in [50]:
            name = f'{res}_005Kusano{C}-{p}'
            hp0['input_shape'] = (res*res,)
            fine_tune(name, res)
            print('\n\n\n\n\n\n\n\n\n\n')

In [ ]:
for key, val in best_evals.items():
    print(f'{key}: {val}')

In [ ]:
with open(f'./working/models_evals.pkl', 'wb') as fn:
    pickle.dump(best_evals, fn)